In [1]:
# importing dependencies
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

#### Preprocessing

In [2]:
# defining constants

img_size = 300

train_dir = 'train/'
validation_dir = 'validation/'

epochs = 5
nb_train_samples = 1024
nb_validation_samples = 1024

In [3]:
# creating labels

# since the pixel values (colours) will be 255, we bring them in range[0, 1].
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='binary')

Found 1024 images belonging to 2 classes.
Found 1024 images belonging to 2 classes.


In [3]:
mobile_net=MobileNetV2(input_shape=(img_size,img_size,3),include_top=False,weights='imagenet')
mobile_net.trainable=False

global_average_layer=tf.keras.layers.GlobalAveragePooling2D()

model=tf.keras.models.Sequential()
model.add(mobile_net)
model.add(global_average_layer)
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))

/Users/rjagtap/opt/anaconda3/envs/tf/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [4]:
mobile_net.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 301, 301, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 150, 150, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 150, 150, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
activation_1 (Activation)    (None, 298, 298, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
activation_2 (Activation)    (None, 147, 147, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 69, 69, 64)        51264     
__________

In [8]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=['accuracy'])

In [9]:
model.fit_generator(
    generator=train_generator, epochs=1, validation_data=validation_generator, 
    steps_per_epoch=nb_train_samples, validation_steps=nb_validation_samples
)

Epoch 1/1
  18/1024 [..............................] - ETA: 24:19 - loss: 0.6989 - acc: 0.5052

KeyboardInterrupt: 